# Class preparation Data

## INSEE

Les données sources de l'INSEE proviennent de [Data Gouv](https://www.data.gouv.fr/fr/datasets/base-sirene-des-entreprises-et-de-leurs-etablissements-siren-siret/)

- communes_insee:
    - Le fichier source pour les communes se trouvent à cette [URL](https://www.insee.fr/fr/information/3720946)
    - Le notebook pour reconstituer le csv est dispon
- upper_word:
- voie:

## INPI 

Les données de l'INPI proviennent de ses différents Notebooks:

- inpi_etb


In [ ]:
from inpi_insee import preparation_data

%load_ext autoreload
%autoreload 2

In [ ]:
param = {
    'communes_insee': r'data\input\Parameters\communes_france.csv',
    'upper_word':r'data\input\Parameters\upper_stop.csv',
     "voie":r'data\input\Parameters\voie.csv',
    'insee': r"data\RawData\INSEE\Stock\ETS\StockEtablissement_utf8.csv",
    'inpi_etb': r"data\RawData\INPI\Stock\initial_ETS.gz",
    'date_end':"2018-01-01"
}
prep_data = preparation_data.preparation(param)

In [ ]:
%%time
prep_data.normalize_inpi(save_gz = True, save_sql = True)

In [ ]:
%%time
prep_data.normalize_insee(siren_inpi_gz = 
                          r'data\input\SIREN_INPI\inpi_etb_stock_4824158.gz',
                                save_gz = True, save_sql = True)

# Test

In [ ]:
test.head()

In [ ]:
test.groupby('len_digit_address_insee')['len_digit_address_insee'].count()

In [ ]:
test.loc[lambda x: 
         x['siren'].isin(['752085324'])
         & (len(x['special_digit'])>=2)
        ]

In [ ]:
test.assign(
        special_digit = lambda x:
        x['libelleVoieEtablissement'].str.findall(r"(\d+)").apply(
        lambda x:'&'.join([i for i in x]))
        )